In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np

# load model
model = tf.keras.models.load_model("/content/drive/Shareddrives/GP/Final/Alx1/CNN_Final_Alx1.h5")

In [3]:
#onehot_decoder
# (38, 4)로 이루어진 벡터 시퀀스를 문자열 시퀀스로 디코딩 한다.
def onehot_decoder(seq, word=['A','G','C','T']):
    out = ""
    
    for i in range(9, 29):
        arg = np.argmax(seq[i])
        out += word[arg]
    #out+="/"        
    return out

In [4]:
#Genetic Algorithm

#Initialize
def init():
  pool = []
  for i in range(30):   #유전자풀에 30개의 초기 염기서열을 (38, 4) 크기의 One-Hot Vector로 생성한다.
    seq = []
    for i in range(9):
        seq.append([.25, .25, .25, .25])
    for i in range(20):
        acid = [0, 0, 0, 0]
        n = np.random.randint(0,4)
        acid[n] = 1
        seq.append(acid)
    for i in range(9):
        seq.append([.25, .25, .25, .25])
    pool.append(seq)
  return np.array(pool)
 
#fitness
def fitness(model, pool):   # model = classifier
  scores = [0]*30
  for i in range(30):       # 유전자 풀의 30개의 염기서열을 분류기로 평가하여 그 점수들을 같은 index를 갖는 scores 리스트로 반환한다.  
    seq = np.array(pool[i]).reshape(1,38,4)
    proba = model.predict(seq)
    scores[i] = proba[0][0]
  
  return scores
 
#selection
def selection(pool, scores):
  temp = []
  while len(temp) < 8:  #적합도가 가장 높은 8개의 유전자를 선정
    index = np.argmax(scores)
    if pool[index] not in temp:
      temp.append(pool[index])
    del scores[index]
    del pool[index]
 
  p1 = temp[0][:] #4개의 부모 유전자
  p2 = temp[1][:]
  p3 = temp[2][:]
  p4 = temp[3][:]
  x = np.random.randint(9,29) #교차 위치 선정
  c1 = p1[:x] + p2[x:] #4개의 자식 유전자 생성
  c2 = p2[:x] + p1[x:]
  x = np.random.randint(9,29)
  c3 = p3[:x] + p4[x:]
  c4 = p4[:x] + p3[x:]
  temp.append(c1)
  temp.append(c2)
  temp.append(c3)
  temp.append(c4)
 
  n = np.random.randint(10,16)  #부모 및 자식의 돌연변이 갯수 선정
  for i in range(n):
    index = np.random.randint(0,12)  #오리지날 부모 및 자식 중 하나 선택
    temp_seq = temp[index][:]
    pos_a = np.random.randint(9,29) #돌연변이 시작위치 선정
    pos_b = np.random.randint(pos_a,29) #돌연변이 끝 위치 선정
    for j in range(pos_a, pos_b+1): #pos_a 부터 pos_b까지의 모든 아미노산을 돌연변이 시킴
      temp_acid = [0, 0, 0, 0]
      r = np.random.randint(0,4)  #돌연변이 값 (A,G,T,C 중 하나) 선정
      temp_acid[r] = 1
      temp_seq[j] = temp_acid[:]
 
    temp.append(temp_seq)
 
  for i in range(18-n): #부모와 자식, 돌연변이 외 나머지는 완전 무작위로 sequence를 생성
    seq = []
    for i in range(9):
        seq.append([.25, .25, .25, .25])
    for i in range(20):
        acid = [0, 0, 0, 0]
        n = np.random.randint(0,4)
        acid[n] = 1
        seq.append(acid)
    for i in range(9):
        seq.append([.25, .25, .25, .25])

    temp.append(seq)
 
  return np.array(temp)

In [5]:
from collections import defaultdict
from tqdm import tqdm

seqs = []
for i in tqdm(range(50)):
  pool = init()
  for j in range(30):
    scores = fitness(model, pool)
    pool = selection(pool.tolist(), scores)[:]
  
  scores = fitness(model, pool)
  for seq, score in zip(pool, scores):
    if score > 0.8:
      most_seq = seq
      decode_seq = onehot_decoder(most_seq)
      seqs.append(decode_seq)


100%|██████████| 50/50 [34:47<00:00, 41.74s/it]


In [10]:
print(len(seqs), len(scores))

995 30


In [ ]:
'''
f = open("/content/drive/Shareddrives/GP/Final/Result Seq/DeepBind_AR_FL.txt", 'w')
for seq in seqs:
  f.write(f'{seq}\n')
f.close()
'''